# Imports

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from PIL import Image

# Datasets Loading

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=True
)

# Device Setting

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Creation

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):           # x.shape = N, 3, 32, 32
        x = F.relu(self.conv1(x))   # x.shape = N, 16, 30, 30
        x = self.pool(x)            # x.shape = N, 16, 15, 15
        x = F.relu(self.conv2(x))   # x.shape = N, 32, 13, 13
        x = self.pool(x)            # x.shape = N, 32, 6, 6
        x = F.relu(self.conv3(x))   # x.shape = N, 64, 4, 4
        x = self.pool(x)            # x.shape = N, 64, 2, 2
        x = torch.flatten(x, 1)     # x.shape = N, 64 * 2 * 2
        x = F.relu(self.fc1(x))     # x.shape = N, 128
        x = F.relu(self.fc2(x))     # x.shape = N, 64
        x = self.fc3(x)             # x.shape = N, 10
        return x

model = Net().to(device)

# Loss and Optimizer

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

# Model Training

In [8]:
model.train()

for epoch in range(5):
    for batch_index, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = loss_fn(output, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (batch_index + 1) % 300 == 0:
            print (f'Epoch [{epoch + 1}], Step [{batch_index + 1}], Loss: {loss.item():.4f}')

Epoch [1], Step [300], Loss: 1.5775
Epoch [1], Step [600], Loss: 1.6139
Epoch [1], Step [900], Loss: 1.5372
Epoch [1], Step [1200], Loss: 1.4639
Epoch [1], Step [1500], Loss: 1.5969
Epoch [2], Step [300], Loss: 1.5465
Epoch [2], Step [600], Loss: 1.4596
Epoch [2], Step [900], Loss: 1.0901
Epoch [2], Step [1200], Loss: 1.0867
Epoch [2], Step [1500], Loss: 1.1851
Epoch [3], Step [300], Loss: 1.3545
Epoch [3], Step [600], Loss: 0.9435
Epoch [3], Step [900], Loss: 0.9230
Epoch [3], Step [1200], Loss: 1.3195
Epoch [3], Step [1500], Loss: 0.8694
Epoch [4], Step [300], Loss: 0.9325
Epoch [4], Step [600], Loss: 1.2398
Epoch [4], Step [900], Loss: 0.7962
Epoch [4], Step [1200], Loss: 0.9552
Epoch [4], Step [1500], Loss: 0.9602
Epoch [5], Step [300], Loss: 1.1499
Epoch [5], Step [600], Loss: 1.0769
Epoch [5], Step [900], Loss: 1.2429
Epoch [5], Step [1200], Loss: 0.8647
Epoch [5], Step [1500], Loss: 1.1268


# Model Testing

In [9]:
model.eval()

with torch.no_grad():
    n_samples = len(test_loader.dataset)
    n_correct = 0

    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(targets)

        output = model(data)

        _, predicted = torch.max(output, 1)

        n_correct += (predicted == targets).sum().item()

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the Net: {accuracy}%')

Accuracy of the Net: 67.22%


In [ ]:
torch.save(model.state_dict(), 'net.pth')

In [ ]:
model = Net()
model.load_state_dict(torch.load('net.pth'))
model.to(device)